## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2023-08-16 23:38:44.751386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
applic_counts = application_df["APPLICATION_TYPE"].value_counts()
applic_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(applic_counts[applic_counts<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df["CLASSIFICATION"].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts_1 = class_counts.loc[class_counts > 1]
class_counts_1


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [35]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [36]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df)

In [37]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop(['IS_SUCCESSFUL'], axis=1)
y = dummies_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [38]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [93]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
first_layer = 80
second_layer = 30
third_layer = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=first_layer, activation="relu", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=second_layer, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=third_layer, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_67 (Dense)            (None, 80)                3520      
                                                                 
 dense_68 (Dense)            (None, 30)                2430      
                                                                 
 dense_69 (Dense)            (None, 10)                310       
                                                                 
 dense_70 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6271 (24.50 KB)
Trainable params: 6271 (24.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [94]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [95]:
# Train the model
fit_nn = nn.fit(X_train_scaled, y_train, epochs=80)

Epoch 1/80
804/804 [==============================] - 2s 1ms/step - loss: 0.5699 - accuracy: 0.7224
Epoch 2/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5552 - accuracy: 0.7315
Epoch 3/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5515 - accuracy: 0.7322
Epoch 4/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5504 - accuracy: 0.7343
Epoch 5/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5500 - accuracy: 0.7338
Epoch 6/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5488 - accuracy: 0.7332
Epoch 7/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5474 - accuracy: 0.7343
Epoch 8/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7347
Epoch 9/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7353
Epoch 10/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5465 - accuracy: 0.7360

In [97]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5873 - accuracy: 0.7269 - 244ms/epoch - 912us/step
Loss: 0.5873030424118042, Accuracy: 0.7268804907798767


In [118]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
first_layer = 50
second_layer = 25
third_layer = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=first_layer, activation="sigmoid", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=second_layer, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=third_layer, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_85 (Dense)            (None, 50)                2200      
                                                                 
 dense_86 (Dense)            (None, 25)                1275      
                                                                 
 dense_87 (Dense)            (None, 10)                260       
                                                                 
 dense_88 (Dense)            (None, 1)                 11        
                                                                 
Total params: 3746 (14.63 KB)
Trainable params: 3746 (14.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [119]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [120]:
# Train the model
fit_nn = nn.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5828 - accuracy: 0.7170
Epoch 2/25
804/804 [==============================] - 1s 2ms/step - loss: 0.5610 - accuracy: 0.7290
Epoch 3/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5572 - accuracy: 0.7313
Epoch 4/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5553 - accuracy: 0.7295
Epoch 5/25
804/804 [==============================] - 1s 2ms/step - loss: 0.5546 - accuracy: 0.7311
Epoch 6/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7311
Epoch 7/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5519 - accuracy: 0.7327
Epoch 8/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5517 - accuracy: 0.7318
Epoch 9/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5508 - accuracy: 0.7326
Epoch 10/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5501 - accuracy: 0.7320

In [121]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5501 - accuracy: 0.7277 - 271ms/epoch - 1ms/step
Loss: 0.5501341223716736, Accuracy: 0.7276967763900757


In [148]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
first_layer = 10
second_layer = 8
third_layer = 6
fourth_layer = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=first_layer, activation="relu", input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=second_layer, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=third_layer, activation="tanh"))

nn.add(tf.keras.layers.Dense(units=fourth_layer, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_118 (Dense)           (None, 10)                440       
                                                                 
 dense_119 (Dense)           (None, 8)                 88        
                                                                 
 dense_120 (Dense)           (None, 6)                 54        
                                                                 
 dense_121 (Dense)           (None, 4)                 28        
                                                                 
 dense_122 (Dense)           (None, 1)                 5         
                                                                 
Total params: 615 (2.40 KB)
Trainable params: 615 (2.40 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [149]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [150]:
# Train the model
fit_nn = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 1ms/step - loss: 0.6039 - accuracy: 0.6885
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5701 - accuracy: 0.7228
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5612 - accuracy: 0.7268
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5577 - accuracy: 0.7282
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5550 - accuracy: 0.7294
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7291
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5526 - accuracy: 0.7298
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5520 - accuracy: 0.7315
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5513 - accuracy: 0.7310
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5515 - accuracy: 0.7314

In [151]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5540 - accuracy: 0.7268 - 274ms/epoch - 1ms/step
Loss: 0.5539889335632324, Accuracy: 0.7267638444900513


In [154]:
# Export our model to HDF5 file
path = "/Users/arminearutyunyan/Desktop/Starter_Code/AlphabetSoupCharity_Optimization.h5"
nn.save(path, save_format="h5")